# NeuroHack Memory System Demo 🧠

This notebook demonstrates the end-to-end pipeline of the `neurohack-memory` system, showcasing:
1. **Configuration**: Setting up the system.
2. **Memory Extraction**: Using LLMs (Groq) or Regex.
3. **Hybrid Search**: Combining Semantic + BM25.
4. **Reranking**: Multi-signal scoring.
5. **Conflict Resolution**: Handling constraints.

In [ ]:
import os
import asyncio
from neurohack_memory import MemorySystem
from neurohack_memory.utils import load_yaml

# Ensure we are in the project root
if os.path.exists("config.yaml"):
    cfg = load_yaml("config.yaml")
else:
    raise FileNotFoundError("config.yaml not found. Run from project root!")

# Initialize System
sys = MemorySystem(cfg)
print("✓ Memory System Initialized")

## 1. Process a Turn (Extraction)
Simulate a user stating preferences.

In [ ]:
user_input = "Hi! My preferred language is Spanish. Also, don't call me on weekends."
print(f"User: {user_input}")

# Process turn (Extracts memories)
result = await sys.process_turn(user_input)

print(f"\nExtracted {len(result['extracted'])} memories:")
for m in result['extracted']:
    print(f"- [{m.type.value}] {m.key}={m.value} (conf={m.confidence})")

## 2. Retrieval with Hybrid Search & Reranking
Simulate a query much later in the conversation.

In [ ]:
query = "Can I schedule a call for Saturday?"
print(f"Future Query: {query}")

# Retrieve relevant memories
retrieval = sys.retrieve(query)

print(f"\nRetrieval Results ({len(retrieval['retrieved'])} hits):")
for i, rm in enumerate(retrieval['retrieved'], 1):
    print(f"{i}. Score: {rm.score:.3f} | {rm.memory.key}={rm.memory.value}")

print(f"\nInjected Context:\n{retrieval['injected_context']}")

## 3. Conflict Resolution Check
What happens if the user changes their mind?

In [ ]:
# User updates preference
update_input = "Actually, I prefer French now."
await sys.process_turn(update_input)

# Query for language
q2 = "What language should I speak?"
r2 = sys.retrieve(q2)

print(f"Query: {q2}")
print("Top Result (Should be French):")
if r2['retrieved']:
    top = r2['retrieved'][0].memory
    print(f"[{top.type.value}] {top.key}={top.value}")
else:
    print("No memory found.")

In [ ]:
sys.close()